In [46]:
import scipy.io
import os
import numpy as np
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')

In [47]:
def get_points(file):
    '''load matlab style file'''
    mat = scipy.io.loadmat(file)
    return mat[list(mat.keys())[3]]

def pickle_save(filename, content):
    '''save the file into python pickle object under output folder'''
    with open(filename, 'wb') as f:
        pickle.dump(content, f)
        
def pickle_open(filename):
    '''load the pickle file'''
    with open(filename, 'rb') as f:
        content = pickle.load(f)
    return content

In [48]:
dir_list =  os.listdir('test_set_predict/points')
dir_list.sort()

fiducial_pt_full = np.stack((get_points('test_set_predict/points/'+filename) for filename in dir_list))



In [49]:
# calculate the pairwise distances
from sklearn.metrics import pairwise_distances

# extract pairwise distance as features (78*77/2=3303 features)
# nrow=number of records of the dataset; ncol=3303
feature_test = np.stack((pairwise_distances(fiducial_pt_full[i])[np.triu_indices(78, k = 1)] for i in range(fiducial_pt_full.shape[0])))

### Not sure how to deal with the warning. Distances are calculated normally though

In [50]:
# Load the baseline model and predict
baseline_model= pd.read_pickle('gbt_baseline.pkl')
result_baseline= baseline_model.predict(feature_test)

In [51]:
# Load the NN model and predict 
import tensorflow as tf
import tensorflow_hub as hub
export_path_keras = "NNmodel.h5"
NN_model = tf.keras.models.load_model(
  export_path_keras, 
  # `custom_objects` tells keras how to load a `hub.KerasLayer`
  custom_objects={'KerasLayer': hub.KerasLayer})

pred_prob = NN_model.predict(feature_test).reshape(len(feature_test))
result_NN = np.zeros(len(feature_test))
result_NN[pred_prob>0.5] = 1


array([1., 1., 1., ..., 0., 0., 1.])

In [72]:

results_final = pd.DataFrame([pd.Series(index=np.arange(1,3001),data=result_baseline).rename('Baseline'),
              pd.Series(index=np.arange(1,3001),data=result_NN).rename('Advanced')]).transpose()
results_final.rename_axis(['Index'],inplace = True)
results_final

,Baseline,Advanced
Index,,
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,1.0
5,1.0,1.0
...,...,...
2996,0.0,0.0
2997,0.0,0.0
2998,0.0,0.0


In [75]:
%cd '/Users/raymondliang/Desktop/test_set_predict'
results_final.to_csv('label_prediction.csv')

/Users/raymondliang/Desktop/test_set_predict
